# Setup
Make sure you followed the instructions in the "Compute guess numbers on plaintext passwords" section in the readme file.

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys, importlib, pprint
import tensorflow as tf
import numpy as np

import UNCM
from input_pipeline import make_dataset
from tester import Tester
from inference import ancestral_sampling


setup_module = 'configs.UNCM_medium_8096con_2048pm'
name_run = 'UNCM_medium_8096con_2048pm'

setup = importlib.import_module(setup_module)
hparams = setup.hparams  

input_fn = UNCM.make_get_input_tensors(hparams)

# load pre-trained UNCM
uncm = UNCM.import_models(hparams, name_run)
conf_encoder_uncm, cpassmodel_uncm = uncm

t = Tester(conf_encoder_uncm, cpassmodel_uncm, input_fn, hparams)

In [2]:
path = 'examples/findfriendz.com__NOHASH__Social.txt'
(password, probability), seed = t.compute_probability_from_file(path, return_X=True)

pp_uncm = list(sorted(zip(np.round(probability, 6), password), key=lambda x: -x[0]))
print(*pp_uncm[:100], sep='\n')

Actual number of users sampled for SEED computation:  8192 

(0.000828, 'password')
(0.000443, 'indian')
(0.000386, 'welcome')
(0.000364, 'krishna')
(0.000357, 'qwerty')
(0.000333, 'chinnu')
(0.000297, 'sairam')
(0.000294, 'saibaba')
(0.000286, 'hanuman')
(0.000286, 'pakistan')
(0.000273, 'iloveyou')
(0.000242, 'aditya')
(0.000232, 'abhishek')
(0.000229, 'sanjay')
(0.000228, 'manisha')
(0.000228, 'mahesh')
(0.000215, 'jaimatadi')
(0.000214, 'sharma')
(0.000204, 'loveyou')
(0.000203, 'bharat')
(0.000201, 'anuradha')
(0.000198, 'abc123')
(0.000197, 'computer')
(0.000196, 'mylove')
(0.000191, 'bismillah')
(0.000186, 'mother')
(0.000185, 'vikram')
(0.000184, 'information')
(0.000182, 'chirag')
(0.000182, 'zxcvbnm')
(0.000181, 'sachin')
(0.000178, 'karthik')
(0.000177, 'london')
(0.000172, 'sweety')
(0.000168, 'master')
(0.000168, 'aaaaaa')
(0.000167, 'santosh')
(0.000166, 'ganesh')
(0.000164, 'lovely')
(0.000164, 'prakash')
(0.000164, 'shalom')
(0.000164, 'anusha')
(0.000156, 'success')
(0

# For non-UNCM (baseline)
The same goes for standard models. The only differece is that the model does not exploit the email addresses to adapt to the target.

In [3]:
setup_module = 'configs.baseline'
name_run = 'baseline'

setup = importlib.import_module(setup_module)
hparams_baseline = setup.hparams  

input_fn = UNCM.make_get_input_tensors(hparams_baseline)

# load pre-trained UNCM
_, passmodel_baseline = UNCM.import_models(hparams_baseline, name_run)

t_baseline = Tester(None, passmodel_baseline, input_fn, hparams_baseline)

In [4]:
path = 'examples/findfriendz.com__NOHASH__Social.txt'
(password, probability), _ = t_baseline.compute_probability_from_file(path, return_X=True)

pp_baseline = list(sorted(zip(np.round(probability, 6), password), key=lambda x: -x[0]))
print(*pp_baseline[:100], sep='\n')

(0.000491, 'qwerty')
(0.000275, 'password')
(0.000125, 'azerty')
(0.00012, 'qwerty123')
(0.000109, 'abc123')
(0.000102, 'loulou')
(0.0001, 'daniel')
(9.8e-05, 'r123456')
(9.1e-05, 'india123')
(8.9e-05, 'welcome123')
(8.8e-05, 'iloveyou')
(8.7e-05, 'ferrari')
(8.4e-05, 'abcd1234')
(8.2e-05, 'killer')
(7.9e-05, 'zxcvbnm')
(7.6e-05, 'dragon')
(7.4e-05, 'martin')
(7.3e-05, 'fuckyou')
(7.2e-05, 'alexander')
(6.6e-05, 'aaaaaa')
(6.5e-05, 'master')
(6.5e-05, 'robert')
(6.4e-05, 'samsung')
(6.1e-05, 'computer')
(6.1e-05, 'asdfghjkl')
(5.9e-05, 'thomas')
(5.7e-05, 'freedom')
(5.6e-05, 'junior')
(5.6e-05, 'liverpool')
(5.6e-05, 'family')
(5.6e-05, 'q1w2e3r4')
(5.6e-05, 'welcome')
(5.3e-05, 'michelle')
(5.2e-05, '123qwe')
(5e-05, 'christian')
(4.9e-05, 'princess')
(4.8e-05, 'sunshine')
(4.7e-05, 'anthony')
(4.7e-05, 'qwertyuiop')
(4.7e-05, 'asdf1234')
(4.6e-05, 'rachel')
(4.6e-05, 'victor')
(4.4e-05, 'a123456789')
(4.4e-05, 'arsenal')
(4.2e-05, 'shadow')
(4.2e-05, 'a1b2c3d4')
(3.9e-05, 'flower')


# Comparison of the two rankings

In [5]:
import pandas as pd

pd.set_option('display.max_rows', None)

df = pd.DataFrame()
df['baseline'] = [x[1] for x in pp_baseline]
df['UNCM'] = [x[1] for x in pp_uncm]
df.head(100)

,baseline,UNCM
0,qwerty,password
1,password,indian
2,azerty,welcome
3,qwerty123,krishna
4,abc123,qwerty
5,loulou,chinnu
6,daniel,sairam
7,r123456,saibaba
8,india123,hanuman
9,welcome123,pakistan
